In [22]:
# MongoDB connection
import pymongo

client = pymongo.MongoClient('mongodb://152.46.20.123:27017')
db = client.testgridfs
collection = db.users
data = []
result = collection.find()
for obj in collection.find():
    if(len(obj['videosViewed']) >= 1):
        for viewed in obj['videosViewed']:
            data.append((obj['userId'], viewed['videoId'], 1))
print (data)

[(2, 1, 1), (2, 6, 1), (2, 7, 1), (2, 2, 1), (2, 3, 1), (2, 4, 1), (2, 5, 1), (2, 6, 1), (2, 7, 1), (2, 8, 1), (2, 9, 1), (2, 10, 1), (3, 1, 1), (3, 7, 1), (4, 1, 1), (4, 7, 1), (4, 4, 1), (4, 5, 1), (4, 10, 1), (5, 6, 1), (5, 7, 1), (5, 10, 1), (6, 1, 1), (6, 7, 1), (6, 2, 1), (6, 4, 1), (6, 8, 1), (7, 1, 1), (7, 6, 1), (7, 5, 1), (7, 6, 1), (7, 7, 1), (7, 8, 1), (7, 10, 1), (8, 1, 1), (8, 10, 1), (9, 5, 1), (9, 6, 1), (9, 7, 1), (9, 8, 1), (10, 1, 1), (10, 6, 1), (10, 7, 1), (10, 9, 1)]


In [1]:
# Retrieve Data
# Data format. Each item in the list represents userid, video_id, num of times viewed / rating
user_ad_views = [[1,1,5],
    [3,2,1],
                 [3,1,1],
                 [1,2,10],
                 [2,4,1],
       [2,1,6],
       [2,2,2],
       [3,3,1]]
user_ad_views = sc.parallelize(user_ad_views)
user_ad_views = user_ad_views.map(lambda x:(x[0], x[1], x[2]))
# print (user_ad_views.collect())

[(1, 1, 5), (3, 2, 1), (3, 1, 1), (1, 2, 10), (2, 4, 1), (2, 1, 6), (2, 2, 2), (3, 3, 1)]


In [2]:
ad_names = [(1,"one"),(2,"two"),(3,"three"),(4,"four")]
ad_names = sc.parallelize(ad_names)

In [3]:
# Data split for cross validation
trainData, validationData, testData = user_ad_views.randomSplit([40, 40, 20], 13)
print (trainData.collect())

[(3, 2, 1), (2, 4, 1), (2, 1, 6), (2, 2, 2)]


In [4]:
unique_user = user_ad_views.map(lambda x:(x[0],(x[1], x[2]))).groupByKey().map(lambda x:x[0])
unique_ad = ad_names.map(lambda x:x[0])
all_pairs = unique_user.cartesian(unique_ad)
trainData_m = trainData.map(lambda x:(x[0], x[1]))
print (unique_user.take(5))
print (unique_ad.take(5))
print (all_pairs.take(5))
print (trainData_m.take(5))

[1, 2, 3]
[1, 2, 3, 4]
[(1, 1), (1, 2), (1, 3), (1, 4), (2, 1)]
[(3, 2), (2, 4), (2, 1), (2, 2)]


In [5]:
# Evaluating a model

def computeAvg(rdd):
    userid, pre, to_c = rdd[0], list(rdd[1][0]), list(rdd[1][1])
    to_c = sc.parallelize(to_c).map(lambda x:x[0]).collect()
    X = len(to_c)
    pre = sc.parallelize(pre).sortBy(lambda x:-x[1]).map(lambda x:x[0]).collect()[:X]
    same = 0
    for i in pre:
        if i in to_c: same+=1
    return same*1.0/X
        
    
def modelEval(model, validationData):
    to_be_predicted = all_pairs.subtract(trainData_m)
    predicted_data = model.predictAll(to_be_predicted).map(lambda x:(x[0], (x[1], x[2]))).groupByKey()
    to_compare = validationData.subtract(trainData).map(lambda x:(x[0], (x[1], x[2]))).groupByKey()
    merge = predicted_data.join(to_compare)
    total = 0
    for each in merge.collect():
        total+=computeAvg(each)
    return (total*1.0/50)

In [6]:
# Building the model
from pyspark.mllib.recommendation import *
import random
from operator import *

for rank in [1, 2, 3]:
    model = ALS.trainImplicit(trainData, rank = rank, seed = 345)
    modelScore = modelEval(model, validationData)
    print "The model score for rank %d is %f"%(rank, modelScore)

The model score for rank 1 is 0.010000
The model score for rank 2 is 0.010000
The model score for rank 3 is 0.010000


In [7]:
bestModel = ALS.trainImplicit(trainData, rank=1, seed=345)
ans = modelEval(bestModel, testData)
print "Model Score for best model, rank = 10 is %f"%(ans)

Model Score for best model, rank = 10 is 0.000000


In [12]:
# Recommend now
top = bestModel.recommendProducts(2, 2)
print (top)

[Rating(user=2, product=2, rating=1.183594083101525), Rating(user=2, product=1, rating=0.8464387884257007)]
